## Load modules

In [3]:
%load_ext autoreload
%matplotlib inline

In [ ]:
!pip install torch
!pip install tensorboardx

In [ ]:
!pip install --pre pytorch-ignite

In [ ]:
!pip install trimesh

In [ ]:
import ignite
ignite.__version__

In [4]:
import trimesh
import trimesh.voxel.creation

C:\Users\Alexandra\ANACONDA3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Alexandra\ANACONDA3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Alexandra\ANACONDA3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
%autoreload 2

from IPython import display
from utils import Logger 

import torch
from torch import nn, optim
from torch.autograd.variable import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

C:\Users\Alexandra\ANACONDA3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [6]:
import scipy.ndimage as nd
import scipy.io as io

C:\Users\Alexandra\ANACONDA3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [12]:
import os

In [ ]:
from ignite.contrib.handlers import ProgressBar
from ignite.engine import Engine, Events
from ignite.handlers import ModelCheckpoint, Timer
from ignite.metrics import RunningAverage

Select device

In [7]:
if torch.cuda.is_available(): 
    print('available')
    device = torch.device("cuda:0")
else:
    print('no cuda available')

available


##  Preprocess the dataset

In [43]:
def getVoxels(pth, cube_size = 64):
    voxels = io.loadmat(pth)['instance']
    voxels = np.pad(voxels,(1,1),'constant',constant_values=(0,0))
    if cube_size !=32 and cube_size==64:
        voxels=nd.zoom(voxels,(2,2,2),mode='constant',order=0)
    return voxels

In [44]:
class ShapeDataset(Dataset):
    def __init__(self,root,cube_size):
        self.root=root
        self.listdir = os.listdir(self.root)
        self.cube_size = cube_size
        
    def __getitem__(self,index):
        with open(os.path.join(self.root,self.listdir[index]),"rb") as f:
            volume = np.asarray(getVoxels(f,self.cube_size),dtype=np.float32)
            return torch.FloatTensor(volume)
        
    def __len__(self):
        return len(self.listdir)

In [45]:
dataset = ShapeDataset('./stairs/30',64)

In [46]:
data_loader = torch.utils.data.DataLoader(dataset,1)

In [47]:
data_loader

## My Networks

In [48]:
class DiscriminatorNet(torch.nn.Module):
    
    #3 hidden-layer discriminative nn
    
    def __init__(self, cube_size = 64):
        super(DiscriminatorNet, self).__init__()
        self.cube_size = cube_size

        padd = (0,0,0)
        #if self.cube_size == 32:
            #padd = (1,1,1)
        
        self.conv1 = nn.Sequential(
            torch.nn.Conv3d(1, self.cube_size, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.BatchNorm3d(self.cube_size),
            torch.nn.LeakyReLU(0.2)
        )
        
        self.conv2 = nn.Sequential(
            torch.nn.Conv3d(self.cube_size,self.cube_size*2, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.BatchNorm3d(self.cube_size * 2),
            torch.nn.LeakyReLU(0.2)
        )
        
        self.conv3 = nn.Sequential(
            torch.nn.Conv3d(self.cube_size * 2,self.cube_size* 4, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.BatchNorm3d(self.cube_size * 4),
            torch.nn.LeakyReLU(0.2)
        )
        
        self.conv4 = nn.Sequential(
            torch.nn.Conv3d(self.cube_size * 4,self.cube_size* 8, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.BatchNorm3d(self.cube_size * 8),
            torch.nn.LeakyReLU(0.2)
        )

        self.out = nn.Sequential(
            torch.nn.Conv3d(self.cube_size * 8, 1, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.out(x)
        return x
        
print(DiscriminatorNet)

<class '__main__.DiscriminatorNet'>


In [49]:
class GeneratorNet(torch.nn.Module):
    def __init__(self, cube_size = 64, in_chans = 200):
        super(GeneratorNet,self).__init__()
        self.cube_size = cube_size
        #self.z_space = z_space
        self.in_chans = in_chans #latent vector


        pad = (0, 0, 0)
        #if self.cube_size == 32:
            #pad = (1,1,1)

        #other version says bias=bias(?check)
        self.convT_1 = nn.Sequential(
            torch.nn.ConvTranspose3d(self.in_chans, self.cube_size*8, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.BatchNorm3d(self.cube_size*8),
            torch.nn.ReLU()
        )
        
        self.convT_2 = nn.Sequential(
            torch.nn.ConvTranspose3d(self.cube_size*8,self.cube_size*4, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.BatchNorm3d(self.cube_size*4),
            torch.nn.ReLU()
        )
        
        self.convT_3 = nn.Sequential(
            torch.nn.ConvTranspose3d(self.cube_size*4,self.cube_size*2, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.BatchNorm3d(self.cube_size*2),
            torch.nn.ReLU()
        )
        
        self.convT_4 = nn.Sequential(
            torch.nn.ConvTranspose3d(self.cube_size*2,self.cube_size, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.BatchNorm3d(self.cube_size),
            torch.nn.ReLU()
        )
         
        self.out = nn.Sequential(
            torch.nn.ConvTranspose3d(self.cube_size,1, kernel_size=4, stride=2, bias=False, padding=1),
            torch.nn.Sigmoid()
        )
        
    
    def forward(self,x):
        x= x.view(-1, self.in_chans, 1, 1, 1)
        x = self.convT_1(x)
        x = self.convT_2(x)
        x = self.convT_3(x)
        x = self.convT_4(x)
        x = self.out(x)
        return x
    


In [50]:
def noise_3d():
    n = torch.randn(batch_size , z_size, 1,1 )
    if torch.cuda.is_available(): return n.cuda()
    return n

## Model

In [51]:
epochs = 20

In [53]:
class Model(object):
    def __init__(self,name,device,data_loader,latent_dim,cube_size):
        self.name=name
        self.device = device
        self.data_loader = data_loader
        self.latent_dim = latent_dim
        self.cube_size = cube_size
        #assert self.name = '3dgan' 
        self.netG = GeneratorNet(self.latent_dim, self.cube_size)
        self.netG.to(self.device)
        self.netD=DiscriminatorNet(self.cube_size)
        self.netD.to(self.device)
        self.optim_G = None
        self.optim_D = None
        self.sceduler_D = None
        self.criterion = torch.nn.BCELoss()

    def create_optim(self,g_lr, d_lr, alpha=0.5,beta=0.5):
        self.optim_G = torch.optim.Adam(self.netG.parameters(), lr= g_lr, betas=(alpha,beta))
        self.optim_D = torch.optim.Adam(self.netD.parameters(), lr= d_lr, betas=(alpha,beta))
        self.scheduler_D = MultiStepLR(self.optim_D, milestones[500,1000])

    def train(self,epochs,d_loss_thresh,log_interval=100,export_interval=10,out_dir='',verbrose=True):
        self.netG.train()
        self.netD.train()
    
        for epoch in range(epochs):
      #batch_time =time.time()
          for batch_idx, data in enumerate(self.data_loader):
            data = data.to(self.device)

            batch_size=data.shape[0]
            real_label= torch.Tensor(batch_size).uniform_(0.7,1.2).to(self.device)
            real_label= torch.Tensor(batch_size).uniform_(0,0.3).to(self.device)
 
            d_real=self.netD(data)
            d_real = d_real.squeeze()
            d_real_loss=self.criterion(d_real,real_label)

            latent=torch.Tensor(batch_size,self.latent_dim).normal_(0,0.33).to(self.device)

            fake=self.netG(latent)
            d_fake=netD(fake.detach())
            d_fake=d_fake.squeeze()
            d_fake_loss =self.criterion(d_fake,fake_label)

            d_loss=d_real_loss + d_fake_loss

            d_real_acc = torch.ge(d_real.squeeze(), 0.5).float()
            d_fake_acc = torch.le(d_fake.squeeze(), 0.5).float()
            d_acc=torch.mean(torch.cat((d_real_acc,d_fake_acc),0))

        if d_acc <=d_loss_thresh:
            self.netD.zero_grad()
            d_loss.backward()
            self.optim_D.step()

        latent = torch.Tensor(batch_size,self.latent_dim).normal_(0,0.33).to(self.device)
        fake=self.netG(latent)
        d_fake=self.netD(fake)
        d_fake=d_fake.squeeze()
        g_loss = self.criterion(d_fake,real_label)

        self.netD.zero_grad()
        self.netG.zero_grad()
        g_loss.backward()
        self.optim_G.step()

        if epoch%export_interval==0:
            samples=fake.cpu().data[:8].squeeze().numpy()
            utils.save_voxels(samples,out_dir,epoch)
        self.scheduler_D.step() 

In [54]:
model = Model('3dModel',device,data_loader,64,32)
model

In [56]:
model.train(epochs,0.01)

PermissionError: [Errno 13] Permission denied: './stairs/30\\test'